<a href="https://colab.research.google.com/github/suhyun6363/what_your_personal_color/blob/oliveyoung_recommend/olibeyoung_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 수현크롤링
# 리뷰평균점수추가
# 소숫점문제해결(pattern_to_remove)

from selenium import webdriver                 # pip install selenium 설치 필요
                                               # 2023년 기준 selenium ver.4.x 설치됨
from selenium.webdriver.common.by import By    # 4.x 버전 이후 find_element호출 방식이 변경되어 추가
import time                                    # sleep() 함수 사용을 위해 추가
from selenium.webdriver.chrome.options import Options
from collections import OrderedDict
import os
import json
import re
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException



url = 'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo='
data = OrderedDict()

beauty_list = {
  'makeup': {
    '베이스메이크업': {
        '블러셔': '1000001000200010006'
    },
    '립메이크업': {
        '립틴트': '1000001000200060003',
        '립스틱': '1000001000200060004',
        '틴티드_립밤': '1000001000200060001',
        '립글로스': '1000001000200060002'
    },
    '아이메이크업': {
        '아이셰도우': '1000001000200070003'
    }
  }
}

# 브라우저 꺼짐 방지 옵션
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
# 크롬 드라이버 생성
driver = webdriver.Chrome("/Users/yeeunahn/PycharmProjects/oliveyoung/driver/chromedriver")

def clean_product_name(product_name):
    # 대소문자를 무시하고 숫자 + colors 또는 color를 찾아 삭제
    pattern_to_remove = r'\b\d+(\.\d+)?\s*colors?\b|\b\d+(\.\d+)?[gm]|\b[NEW]\b|\b\d+\.\s*|\bl\b'



    # 대소문자 무시 플래그 추가
    product_name = re.sub(pattern_to_remove, '', product_name, flags=re.IGNORECASE)

    patterns_to_remove = [
        r'\([^)]*\)',  # (내용)
        r'\[[^]]*\]',  # [내용]
        r'\d+종',
        r'택\s*\d+',  # 택 + 숫자 (예: 택 1, 택1)
        r'[+]',  # + (예: 1+1)
        r'\d+g',  # 숫자 + g (예: 4g)
        r'\d+\.\d+g',  # 숫자 + 소수점 + 숫자 + g (예: 4.8g)
        r'\d+ml',  # 숫자 + m 또는 M + g 또는 G (예: 4ml)
        r'\d+\.\d+ml',  # 숫자 + 소수점 + 숫자 + g (예: 4.8ml)
        r'AD',
        r'ad',
        r'단품',
        r'기획',
        r'한정기획',
        r'기획세트',
        r'NEW',
        r'/',           # / (예: /)
    ]

    # 각 패턴을 순회하며 삭제
    for pattern in patterns_to_remove:
        product_name = re.sub(pattern, '', product_name)

    # 중복 공백 제거
    product_name = ' '.join(product_name.split())

    return product_name

def get_product_info(small_list, category):
    '''
    category_list : list(String). 해당 상품의 대/중/소/카테고리     # ex) 메이크업 > 립메이크업(small) > 립틴트(category)
    name : String. 상품 이름
    number : String. 상품 고유번호
    brand : String. 브랜드 이름
    img : String (src). 대표 이미지 -> 복수 개일 수 있으므로 변경 필요
    product_img_list : list(String). 대표 이미지들의 리스트
    is_discount : boolean. 할인 여부
    origin_price : String (**원). 정상가
    discount_price : String (**원). 할인 가격
    average_rate : 리뷰평균

    옵션이 없는 단일 상품의 경우, 옵션 개수를 0개로 할 것인가 1개로 할 것인가
    그리고 옵션 이름 목록과 가격에 그냥 name과 price를 넣어야 하나?
    품절의 경우 옵션 가격은 얼마?

    option_count : String. 옵션 개수
    option_name_list : list(String). 옵션별 이름
    option_price_list : list(String). 옵션별 가격
    option_img_list : list(String). 옵션별 이미지 src -> colorchip_list
    '''

    time.sleep(0.5)

    number = driver.find_element_by_class_name('prd_btn_area > .btnZzim') \
        .get_attribute('data-ref-goodsno')
    img = (driver.find_element_by_id('mainImg')).get_attribute('src')
    brand = (driver.find_element_by_class_name('prd_brand')).text
    name = (driver.find_element_by_class_name('prd_name')).text
    # 정규표현식을 사용하여 대괄호와 대괄호 안의 내용 삭제
    name = clean_product_name(name)
    try:
        # WebDriverWait를 사용하여 요소가 나타날 때까지 대기
        average_rate_elem = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#repReview > b'))
        )
        average_rate = average_rate_elem.text
    except (TimeoutException, NoSuchElementException):
        average_rate = None
        print("No average rate found for this product.")

    # average_rate = driver.find_elements_by_css_selector('# repReview > b')
    # repReview > b

    cat = driver.find_elements_by_class_name('loc_history > li > .cate_y')
    category_list = [c.text for c in cat]

    discount_price = (driver.find_element_by_class_name('price-2')).text.split('\n')[0]

    try:
        # 할인 상품인 경우 .price-1 요소가 있음
        origin_price = (driver.find_element_by_class_name('price-1')).text.split('\n')[0]
        is_discount = True
    except:
        # 할인이 아닌 경우 discount_price가 곧 origin_price
        # 즉, 어느 경우든 discount_price가 해당 상품의 최종가
        origin_price = discount_price
        is_discount = False

    origin_price = origin_price.replace(',', '')
    discount_price = discount_price.replace(',', '')

    option_name_list = []
    option_price_list = []
    option_img_list = []

    try:
        # 옵션이 없는 경우 .prd_option_box 요소가 없음 (except로 넘어감)
        # .prd_option_box를 클릭해야 .option_value가 드러남
        driver.find_element_by_class_name('prd_option_box').click()

        # 품절 상품인 경우 .type1 soldout임
        options = driver.find_elements_by_class_name('type1 > a > div > .option_value')

        if not options:  # 옵션에 상품 이미지가 없는 경우 .type1 없이 <li class> 태그임
            # options 자체는 WebElement가 요소인 리스트임. option들을 가진 리스트
            options = driver.find_elements_by_tag_name('li > a > div > .option_value')

        # 옵션명, 가격이 차례로 요소로 들어간 리스트가 option_values가 됨
        # ex) ['04 데일리톡(리뉴얼)', '7,840원']
        # 이 때, 품절 상품의 경우 상품 이름만 요소로 들어감. 옵션 자체에 가격이 없음
        option_values = [option.text.split('\n') for option in options]

        # 상품의 옵션 이름만 리스트로 뽑아옴
        option_name_list = [option[0] for option in option_values]
        option_count = len(options)

        for k in range(len(option_name_list)):
            option_name = option_name_list[k]
            # 품절일 경우, 기본가인 discount_price를 품절 상품 가격으로 둠
            if option_name.find('(품절)') != -1:
                option_price_list.append(discount_price)
            else:
                option_price_list.append(option_values[k][1].rstrip('원'))
        option_name_list = [clean_product_name(option[0]) for option in option_values]

        # 옵션은 있으나 옵션에 이미지가 없는 경우 except로 넘어감
        # 옵션에 이미지가 있는 경우에만 option_img_list에 각 이미지를 넣음
        option_imgs = driver.find_elements_by_class_name('type1 > a > span > img')
        option_img_list = [img.get_attribute('src') for img in option_imgs]

    except:
        # 품절일 때 option_count = 0이 돼서 가격이 안 나옴
        option_count = 0

    data["category_list"] = category_list
    data["name"] = name
    data["number"] = number
    data["brand"] = brand
    data["img"] = img
    data["average_rate"] = average_rate
    data["is_discount"] = is_discount
    data["origin_price"] = origin_price
    data["discount_price"] = discount_price
    data["option_count"] = option_count
    data["option_name_list"] = option_name_list
    data["option_price_list"] = option_price_list
    data["colorchip_list"] = option_img_list


    try:
        if len(data["colorchip_list"]) > 0:
            print(data)
            with open(
                    './data/{0}/{1}/{2}.json'.format(small_list, category, number),
                    'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent="\t")
    except:  # 디렉터리가 없을 때만 디렉터리를 만듦
        os.makedirs('./data/{0}/{1}'.format(small_list, category))

    driver.back()  # 뒤로가기
    time.sleep(0.5)

for small_list in beauty_list['makeup']:  # small_list는 String
    for category in beauty_list['makeup'][small_list]:  # category는 String
        driver.get(url + beauty_list['makeup'][small_list][category])
        driver.implicitly_wait(1)
        count = 0  # count로 몇 번째 item의 정보를 가져올지 정함

        # 상품을 감싼 태그를 빼냄. 24/36/48개
        items = driver.find_elements_by_xpath('//li[@criteo-goods]')

        for index in range(len(items)):
            # 개별 아이템을 고름
            item = driver.find_element_by_xpath(
                '//*[@id="Contents"]/ul[%s]/li[%s]/div/a'
                % ((count // 4) + 2, (count % 4) + 1)
            )
            item.click()
            get_product_info(small_list, category)
            count += 1

        page_index = 0
        # #Container > div.pageing > a:nth-child(2)
        pages = driver.find_elements_by_css_selector('#Container > div.pageing > a')
        pages_count = len(pages)
        if pages_count > 0:
            while True:
                pages = driver.find_elements_by_css_selector('#Container > div.pageing > a')
                pages[page_index].click()
                time.sleep(0.5)
                count = 0  # count로 몇 번째 item의 정보를 가져올지 정함

                # 상품을 감싼 태그를 빼냄. 24/36/48개
                items = driver.find_elements_by_xpath('//li[@criteo-goods]')

                for index in range(len(items)):
                    # 개별 아이템을 고름
                    item = driver.find_element_by_xpath(
                        '//*[@id="Contents"]/ul[%s]/li[%s]/div/a'
                        % ((count // 4) + 2, (count % 4) + 1)
                    )
                    item.click()
                    get_product_info(small_list, category)
                    count += 1

                page_index += 1

                # 마지막 페이지인 경우 종료
                if page_index == pages_count:
                    break

driver.quit()

In [ ]:
# 위에서 크롤링한 색상표 > 클러스터링 후 RGB값을 구하는 코드

import os
import numpy as np
from PIL import Image
import csv
import requests
from io import BytesIO
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import json
import sys
import colorsys
from skimage.color import rgb2lab


# JSON 파일을 읽어서 product_info_list를 생성하는 함수
def read_json_files(base_folder, categories):
    product_info_list = []

    for category in categories:
        category_folder = os.path.join(base_folder, category)
        subcategories = [f for f in os.listdir(category_folder) if os.path.isdir(os.path.join(category_folder, f))]

        for subcategory in subcategories:
            subcategory_folder = os.path.join(category_folder, subcategory)
            json_files = [f for f in os.listdir(subcategory_folder) if
                          f.endswith('.json') and not f.startswith('.DS_Store')]

            for json_file in json_files:
                json_file_path = os.path.join(subcategory_folder, json_file)

                with open(json_file_path, 'r', encoding='utf-8') as file:
                    product_info = json.load(file)
                    product_info_list.append(product_info)

    return product_info_list


# 기존 데이터 예시와 유사한 구조의 폴더 구조
base_folder = "/Users/yeeunahn/PycharmProjects/oliveyoung/data 2"
categories = [
    "립메이크업",
    "베이스메이크업",
    "아이메이크업",
]

# 모든 JSON 파일을 읽어서 product_info_list를 생성
product_info_list = read_json_files(base_folder, categories)


# 이미지 클러스터링 및 RGB 값 계산 함수
def calculate_option_name_rgb(colorchip_url):
    response = requests.get(colorchip_url)
    img = Image.open(BytesIO(response.content))

    # 이미지를 RGB 형식으로 변환 (채널 수가 4인 경우, RGBA에서 A 제거)
    img = img.convert("RGB")

    img_array = np.array(img)

    # [0, 0, 0] 및 [255, 255, 255]인 픽셀 제거
    non_black_and_white_pixels = img_array[~np.all((img_array == [0, 0, 0]) | (img_array == [255, 255, 255]), axis=-1)]

    # 이미지를 2D 배열로 변환
    face_data = non_black_and_white_pixels.reshape((-1, 3))

    if face_data.shape[1] != 3:
        raise ValueError("Unsupported number of color channels")

    # 표준화 (Standardization) - 평균이 0, 표준편차가 1이 되도록 스케일 조정
    scaler = StandardScaler()
    face_data_scaled = scaler.fit_transform(face_data)

    # 최적의 k 값으로 k-means 클러스터링 수행
    optimal_k = 4
    optimal_kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
    optimal_cluster_labels = optimal_kmeans.fit_predict(face_data_scaled)

    # 클러스터 중심값(RGB 형식) 출력
    cluster_centers_rgb = scaler.inverse_transform(optimal_kmeans.cluster_centers_)

    # 가장 픽셀 수가 많은 클러스터 찾기
    most_pixels_cluster = max(Counter(optimal_cluster_labels), key=Counter(optimal_cluster_labels).get)

    # 가장 픽셀 수가 많은 클러스터의 RGB 값 출력
    most_pixels_cluster_rgb = cluster_centers_rgb[most_pixels_cluster]
    print(f'Most Pixels Cluster RGB for option {option_name}: {most_pixels_cluster_rgb}')

    return most_pixels_cluster_rgb.tolist()


# 결과를 저장할 리스트 초기화
results = []

# 이미지 URL을 사용하여 option_name별 RGB 값을 계산하고 결과 리스트에 추가
for product_info in product_info_list:
    option_name_list = product_info.get("option_name_list", [])
    colorchip_list = product_info.get("colorchip_list", [])

    option_name_rgb_list = []
    for option_name, colorchip_url in zip(option_name_list, colorchip_list):
        option_name_rgb = calculate_option_name_rgb(colorchip_url)
        option_name_rgb_list.append({"option_name": option_name, "option_rgb": option_name_rgb})

    results.append({
        "category_list": product_info.get("category_list", []),
        "brand": product_info.get("brand", ""),
        "name": product_info.get("name", ""),
        "option_name_rgb_list": option_name_rgb_list
    })

# 결과를 출력
# print(json.dumps(results, indent=4))

# 결과를 CSV 파일로 저장
csv_file_path = "/Users/yeeunahn/PycharmProjects/oliveyoung/product_rgb_info.csv"
with open(csv_file_path, "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = ["category_list", "brand", "name", "option_name", "option_rgb"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for result in results:
        category_list = result["category_list"]
        brand = result["brand"]
        name = result["name"]
        option_name_rgb_list = result["option_name_rgb_list"]

        for option_info in option_name_rgb_list:
            option_name = option_info["option_name"]
            option_rgb = option_info["option_rgb"]

            writer.writerow({
                "category_list": "/".join(category_list),
                "brand": brand,
                "name": name,
                "option_name": option_name,
                "option_rgb": option_rgb
            })

# 프로그램 종료
sys.exit()


In [ ]:
# RGB를 vbs로 변경
import csv
from colormath.color_objects import sRGBColor, LabColor, HSVColor
from colormath.color_conversions import convert_color

def rgb_to_lab(rgb):
    color_rgb = sRGBColor(rgb[0] / 255.0, rgb[1] / 255.0, rgb[2] / 255.0)
    color_lab = convert_color(color_rgb, LabColor)
    return [color_lab.lab_l, color_lab.lab_b]

def rgb_to_hsv(rgb):
    color_rgb = sRGBColor(rgb[0] / 255.0, rgb[1] / 255.0, rgb[2] / 255.0)
    color_hsv = convert_color(color_rgb, HSVColor)
    return [color_hsv.hsv_v, color_hsv.hsv_s]

def process_csv(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as csv_file:
        reader = csv.DictReader(csv_file)

        header = reader.fieldnames + [ 'b', 'v', 's']

        with open(output_file, 'w', newline='', encoding='utf-8') as output_csv:
            writer = csv.DictWriter(output_csv, fieldnames=header)
            writer.writeheader()

            for row in reader:
                rgb_values = row['option_rgb'][1:-1].split(',')
                rgb = [float(value) for value in rgb_values]

                # Convert RGB to LAB and HSV
                lab_values = rgb_to_lab(rgb)
                hsv_values = rgb_to_hsv(rgb)

                # Add new columns to the row
                row['b'], row['v'], row['s'] = lab_values[1], hsv_values[0], hsv_values[1]

                # Write the updated row to the new CSV file
                writer.writerow(row)

# Example usage
input_csv_file = '/content/oliveyoung_rgb.csv'
output_csv_file = 'oliveyoung.csv'
process_csv(input_csv_file, output_csv_file)


In [ ]:
# 각 카테고리당 구한 무게중심을 기준으로 퍼컬분류

import pandas as pd
import numpy as np

# CSV 파일 경로를 지정합니다
csv_file_path = '/content/oliveyoung.csv'
df = pd.read_csv(csv_file_path)

results = []
for index, row in df.iterrows():
    category_2 = row['category_2']
    v_value, b_value, s_value = row['v'] * 100, row['b'], row['s'] * 100  # v 및 s 값에 100을 곱합니다


    # 무게중심
    if category_2 == '립메이크업':
        v_weight, b_weight, s_weight = 79, 17, 51
    elif category_2 == '베이스메이크업':
        v_weight, b_weight, s_weight = 85, 11, 29
    elif category_2 == '아이메이크업':
        v_weight, b_weight, s_weight = 82, 9, 20
    else:
        v_weight, b_weight, s_weight = 0, 0, 0

    if v_value > v_weight and b_value > b_weight and s_value > s_weight:
        result = "Spring warm bright"
    elif v_value > v_weight and b_value > b_weight and s_value <= s_weight:
        result = "Spring warm light"
    elif v_value > v_weight and b_value <= b_weight and s_value <= s_weight:
        result = "Summer cool light"
    elif v_value <= v_weight and b_value <= b_weight and s_value <= s_weight:
        result = "Summer cool mute"
    elif v_value <= v_weight and b_value > b_weight and s_value <= s_weight:
        result = "Autumn warm mute"
    elif v_value <= v_weight and b_value > b_weight and s_value > s_weight:
        result = "Autumn warm deep"
    elif v_value <= v_weight and b_value <= b_weight and s_value > s_weight:
        result = "Winter cool deep"
    elif v_value > v_weight and b_value <= b_weight and s_value > s_weight:
        result = "Winter cool bright"

    results.append(result)

# 분석 결과를 DataFrame에 추가합니다
df['result'] = results

# 결과를 새로운 CSV 파일로 저장합니다
output_csv_path = 'oliveyoung_analysis.csv'
df.to_csv(output_csv_path, index=False)

print(f"분석이 완료되었고, 결과가 {output_csv_path} 파일에 저장되었습니다.")


In [ ]:
# 리뷰크롤링코드
# 근데 리뷰내용은 크롤링할필요가 없을것같으니까 일단 무시해조..

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from collections import OrderedDict
import os
import json
import re
import pandas as pd
import time
from selenium.common.exceptions import TimeoutException

url = 'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo='

beauty_list = {
    'makeup': {

        '립메이크업': {

            '립글로스': '1000001000200060002'
        },

    }
}

# 브라우저 꺼짐 방지 옵션
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 크롬 드라이버 생성
driver = webdriver.Chrome("/Users/yeeunahn/PycharmProjects/oliveyoung/driver/chromedriver")

# 데이터 프레임 생성
df_review = pd.DataFrame(columns=['brand', 'clean_name', 'category_list', 'id', 'option', 'rate', 'color_development', 'vitality',
                                 'spreadability', 'moisture', 'txt'])


def clean_product_name(product_name):
    # 대소문자를 무시하고 숫자 + colors 또는 color를 찾아 삭제
    pattern_to_remove = r'\d+\s*colors?'

    # 대소문자 무시 플래그 추가
    product_name = re.sub(pattern_to_remove, '', product_name, flags=re.IGNORECASE)

    patterns_to_remove = [
        r'\([^)]*\)',  # (내용)
        r'\[[^]]*\]',  # [내용]
        r'\d+종',
        r'택\s*\d+',  # 택 + 숫자 (예: 택 1, 택1)
        r'[+]',  # + (예: 1+1)
        r'\d+g',  # 숫자 + g (예: 4g)
        r'\d+\.\d+g',  # 숫자 + 소수점 + 숫자 + g (예: 4.8g)
        r'\d+ml',  # 숫자 + m 또는 M + g 또는 G (예: 4ml)
        r'\d+\.\d+ml',  # 숫자 + 소수점 + 숫자 + g (예: 4.8ml)
        r'AD',
        r'ad',
        r'단품',
        r'기획',
        r'한정기획',
        r'기획세트',
        r'NEW',
        r'/',           # / (예: /)
    ]

    # 각 패턴을 순회하며 삭제
    for pattern in patterns_to_remove:
        product_name = re.sub(pattern, '', product_name)

    # 중복 공백 제거
    product_name = ' '.join(product_name.split())

    return product_name


def get_product_info(small_list, category):
    try:
        element_present = EC.presence_of_element_located(
            (By.CSS_SELECTOR, 'a.goods_reputation[data-attr="상품상세^상품상세_SortingTab^리뷰"]'))

        WebDriverWait(driver, timeout=10).until(element_present)

        # '리뷰' 링크 클릭
        driver.find_element(By.CSS_SELECTOR, 'a.goods_reputation[data-attr="상품상세^상품상세_SortingTab^리뷰"]').click()
        WebDriverWait(driver, timeout=10).until(element_present)

        # 총 리뷰 수 가져오기
        total_num_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR,
                                            '#gdasContentsArea > div > div.product_rating_area.review-write-delete > div > div.star_area > p.total > em'))
        )

        total_num_text = total_num_element.text


            # total_num_text에 쉼표가 있으면 제거
        total_num_text = total_num_text.replace(',', '')

        total_num = int(total_num_text)
        print(total_num)

        # 리뷰 크롤링 함수 호출
        page_num = min((total_num // 10) + 1, 100)
        review_crawling(small_list, category, page_num, total_num)

        time.sleep(0.5)

        # brand = (driver.find_element_by_class_name('prd_brand')).text
        # name = (driver.find_element_by_class_name('prd_name')).text
        # # 정규표현식을 사용하여 대괄호와 대괄호 안의 내용 삭제
        # name = clean_product_name(name)
        #
        # cat = driver.find_elements_by_class_name('loc_history > li > .cate_y')
        # category_list = [c.text for c in cat]

        driver.back()  # 뒤로가기
        time.sleep(0.5)
    except Exception as e:
        print(f"Error type: {type(e)}, Message: {str(e)}")


# 상품 리뷰 크롤링 함수
def review_crawling(small_list, category, page_num, total_num):
    for current_page in range(1, page_num+1):

        # 현재 페이지가 마지막 페이지인지 확인
        is_last_page = current_page == page_num

        # 현재 페이지에서 크롤링할 리뷰의 범위 계산
        if is_last_page:
            remaining_reviews = total_num % 10  # 마지막 페이지에서 나머지 리뷰 수 계산
            review_range = range(1, remaining_reviews + 1)
        else:
            review_range = range(1, 11)  # 한 페이지당 10개의 리뷰 크롤링

        # 리뷰의 인덱스는 한 페이지 내에서 1~10까지 존재
        for i in review_range:  # 한 페이지 내 10개 리뷰 크롤링
            try:
                brand = (driver.find_element_by_class_name('prd_brand')).text
                name = (driver.find_element_by_class_name('prd_name')).text
                # 정규표현식을 사용하여 대괄호와 대괄호 안의 내용 삭제
                clean_name = clean_product_name(name)

                cat = driver.find_elements_by_class_name('loc_history > li > .cate_y')
                category_list = [c.text for c in cat]

                # 각 요소를 찾을 때 WebDriverWait 사용하여 대기
                id = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.info > div > p.info_user > a'))
                ).text
                option = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) .review_cont p'))
                ).text
                # rate 값을 추출하고 뒤에 점수만 데이터로 저장
                rate = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) .score_area .review_point span'))
                ).text
                # underscore를 기준으로 나누고 마지막 부분 추출
                rate = rate.split('_')[-1]


                color_development = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.poll_sample > dl:nth-child(1) > dd > span'))
                ).text
                vitality = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.poll_sample > dl:nth-child(2) > dd > span'))
                ).text
                spreadability = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.poll_sample > dl:nth-child(3) > dd > span'))
                ).text
                moisture = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.poll_sample > dl:nth-child(4) > dd > span'))
                ).text

                txt = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) .txt_inner'))
                ).text

                # 데이터 프레임에 추가
                df_review.loc[len(df_review)] = [brand, clean_name, category_list, id, option, rate, color_development, vitality, spreadability, moisture, txt]

            except TimeoutException as e:
                print(f"TimeoutException: {str(e)}")

            except Exception as e:
                print(f"Error type: {type(e)}, Message: {str(e)}")

        print(f'{clean_name}, {current_page}페이지 크롤링 완료')

        try:
            if current_page % 10 != 0:  # 현재 페이지가 10의 배수가 아닐 때
                if current_page // 10 < 1:  # 페이지 수가 한 자리수 일 때
                    # 리뷰 10개 긁으면 next 버튼 클릭
                    page_button = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR,
                             f'#gdasContentsArea > div > div.pageing > a:nth-child({current_page % 10 + 1})'))
                    )
                    page_button.click()
                    time.sleep(2)
                else:  # 페이지 수가 두자리 수 이상일 때
                    # 리뷰 10개 긁으면 next 버튼 클릭
                    page_button = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR,
                             f'#gdasContentsArea > div > div.pageing > a:nth-child({current_page % 10 + 2})'))
                    )
                    page_button.click()
                    time.sleep(2)
            else:
                next_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a.next'))
                )
                next_button.click()  # 현재 페이지가 10의 배수일 때 페이지 넘김 버튼 클릭
                time.sleep(2)
                if current_page == page_num:
                    try:
                        prev_button = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located(
                                (By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a.prev'))
                        )
                        prev_button.click()  # 이전 페이지로 돌아가기
                        time.sleep(2)
                    except Exception as e:
                        print(f"Error: {e}")
                    break

        except TimeoutException as e:
            print(f"TimeoutException: {str(e)}")

        except Exception as e:
            print(f"Error type: {type(e)}, Message: {str(e)}")



        except Exception as e:
            print(f"Error type: {type(e)}, Message: {str(e)}")
# 크롤링 시작
for small_list in beauty_list['makeup']:
    for category in beauty_list['makeup'][small_list]:
        driver.get(url + beauty_list['makeup'][small_list][category])
        driver.implicitly_wait(1)
        count = 0  # count로 몇 번째 item의 정보를 가져올지 정함

        # 상품을 감싼 태그를 빼냄. 24/36/48개
        items = driver.find_elements_by_xpath('//li[@criteo-goods]')

        for index in range(len(items)):
            # 개별 아이템을 고름
            item = driver.find_element_by_xpath(
                '//*[@id="Contents"]/ul[%s]/li[%s]/div/a'
                % ((count // 4) + 2, (count % 4) + 1)
            )
            item.click()
            get_product_info(small_list, category)
            count += 1

        page_index = 0
        # #Container > div.pageing > a:nth-child(2)
        pages = driver.find_elements_by_css_selector('#Container > div.pageing > a')
        pages_count = len(pages)
        if pages_count > 0:
            while True:
                pages = driver.find_elements_by_css_selector('#Container > div.pageing > a')
                pages[page_index].click()
                time.sleep(0.5)
                count = 0  # count로 몇 번째 item의 정보를 가져올지 정함

                # 상품을 감싼 태그를 빼냄. 24/36/48개
                items = driver.find_elements_by_xpath('//li[@criteo-goods]')

                for index in range(len(items)):
                    # 개별 아이템을 고름
                    count += 1
                    item = driver.find_element_by_xpath(
                        '//*[@id="Contents"]/ul[%s]/li[%s]/div/a'
                        % ((count // 4) + 2, (count % 4) + 1)
                    )
                    item.click()
                    get_product_info(small_list, category)
                    # count += 1

                page_index += 1

                # 마지막 페이지인 경우 종료
                if page_index == pages_count:
                    break



# 'rate' 컬럼 값 변환 함수
def convert_rate_to_numeric(rate):
    if rate == '5점만점에 5점':
        return 5
    elif rate == '5점만점에 4점':
        return 4
    elif rate == '5점만점에 3점':
        return 3
    elif rate == '5점만점에 2점':
        return 2
    elif rate == '5점만점에 1점':
        return 1
    else:
        return None  # 다른 경우에는 None 또는 원하는 기본값 설정

# 크롤링한 데이터를 CSV 파일로 저장
df_review.to_csv('review_data.csv', index=False)

# 'rate' 컬럼의 값을 숫자로 변환
df_review['rate'] = df_review['rate'].apply(convert_rate_to_numeric)

# 드라이버 종료
driver.quit()

